In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
""" Load the MNIST dataset """
(x_train, _), (_, _) = mnist.load_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
""" Normalize the pixel values to be between -1 and 1 """
x_train = x_train / 127.5 - 1.0
x_train = np.expand_dims(x_train, axis=-1)


In [ ]:
""" Generator model """
generator = Sequential([
    Dense(7*7*128, input_dim=100),
    LeakyReLU(0.2),
    Reshape((7, 7, 128)),
    Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
    LeakyReLU(0.2),
    Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
    LeakyReLU(0.2),
    Conv2D(1, (7, 7), activation='tanh', padding='same')
])

""" Discriminator model """
discriminator = Sequential([
    Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=(28, 28, 1)),
    LeakyReLU(0.2),
    Dropout(0.4),
    Conv2D(128, (3, 3), strides=(2, 2), padding='same'),
    LeakyReLU(0.2),
    Dropout(0.4),
    Flatten(),
    Dense(1, activation='sigmoid')
])

opt = Adam(
    learning_rate=0.0002,
    beta_1=0.5,
)

discriminator.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False

gan = Sequential([generator, discriminator])

gan.compile(optimizer=opt, loss='binary_crossentropy')



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
""" Training the GAN """
batch_size = 64
epochs = 20000

In [ ]:
for epoch in range(epochs):
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_images = generator.predict(noise)

    real_images = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
    x = np.concatenate((real_images, fake_images))

    y_real = np.ones((batch_size, 1))
    y_fake = np.zeros((batch_size, 1))
    y = np.concatenate((y_real, y_fake))

    d_loss = discriminator.train_on_batch(x, y)

    noise = np.random.normal(0, 1, (batch_size, 100))
    y_gen = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch(noise, y_gen)

    if epoch % 1000 == 0:
        print(f"Epoch: {epoch} / {epochs}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch: 0 / 20000, Discriminator Loss: 0.6950635313987732, Generator Loss: [array(0.69506353, dtype=float32), array(0.69506353, dtype=float32), array(0.5234375, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 253ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 244ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
""" Generate synthetic images """
noise = np.random.normal(0, 1, (10, 100))
generated_images = generator.predict(noise)

In [ ]:
""" Plot the generated images """
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow((generated_images[i].reshape(28, 28) + 1) / 2, cmap='gray')
    plt.axis('off')
plt.tight_layout()
plt.show()

## THNAK YOU --- \\(^o^)/